In [1]:
import numpy as np
import pandas as pd

# 1. Categorical

In [2]:
train_cat = pd.read_excel('../../Data/widsdatathon2025/TRAIN/TRAIN_CATEGORICAL_METADATA.xlsx')
train_cat.head()

,participant_id,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ,Barratt_Barratt_P2_Edu,Barratt_Barratt_P2_Occ
0,UmrK0vMLopoR,2016,1,0.0,0,1,21,45,21,45
1,CPaeQkhcjg7d,2019,3,1.0,2,3,15,15,0,0
2,Nb4EetVPm3gs,2016,1,1.0,8,1,18,40,0,0
3,p4vPhVu91o4b,2018,3,0.0,8,3,15,30,18,0
4,M09PXs7arQ5E,2019,3,0.0,1,3,15,20,0,0


## Imputation

In [3]:
print(train_cat.isna().any())

participant_id                      False
Basic_Demos_Enroll_Year             False
Basic_Demos_Study_Site              False
PreInt_Demos_Fam_Child_Ethnicity     True
PreInt_Demos_Fam_Child_Race         False
MRI_Track_Scan_Location             False
Barratt_Barratt_P1_Edu              False
Barratt_Barratt_P1_Occ              False
Barratt_Barratt_P2_Edu              False
Barratt_Barratt_P2_Occ              False
dtype: bool


In [4]:
mode = train_cat['PreInt_Demos_Fam_Child_Ethnicity'].mode()[0]
print(mode)

0.0


In [5]:
train_cat.fillna({'PreInt_Demos_Fam_Child_Ethnicity':mode}, inplace=True)

In [6]:
print(train_cat.isna().any())

participant_id                      False
Basic_Demos_Enroll_Year             False
Basic_Demos_Study_Site              False
PreInt_Demos_Fam_Child_Ethnicity    False
PreInt_Demos_Fam_Child_Race         False
MRI_Track_Scan_Location             False
Barratt_Barratt_P1_Edu              False
Barratt_Barratt_P1_Occ              False
Barratt_Barratt_P2_Edu              False
Barratt_Barratt_P2_Occ              False
dtype: bool


## One-hot

In [7]:
for col in train_cat.select_dtypes(include=['int', 'float']).columns:
    train_cat[col] = train_cat[col].astype('category')

In [8]:
columns_to_encode = train_cat.columns[1:].tolist()
columns_to_encode

['Basic_Demos_Enroll_Year',
 'Basic_Demos_Study_Site',
 'PreInt_Demos_Fam_Child_Ethnicity',
 'PreInt_Demos_Fam_Child_Race',
 'MRI_Track_Scan_Location',
 'Barratt_Barratt_P1_Edu',
 'Barratt_Barratt_P1_Occ',
 'Barratt_Barratt_P2_Edu',
 'Barratt_Barratt_P2_Occ']

In [10]:
train_encoded = pd.get_dummies(train_cat[columns_to_encode], drop_first=True)
train_encoded = train_encoded.map(lambda x: 1 if x is True else (0 if x is False else x))
train_encoded.head()

,Basic_Demos_Enroll_Year_2016,Basic_Demos_Enroll_Year_2017,Basic_Demos_Enroll_Year_2018,Basic_Demos_Enroll_Year_2019,Basic_Demos_Enroll_Year_2020,Basic_Demos_Study_Site_2,Basic_Demos_Study_Site_3,Basic_Demos_Study_Site_4,PreInt_Demos_Fam_Child_Ethnicity_1.0,PreInt_Demos_Fam_Child_Ethnicity_2.0,...,Barratt_Barratt_P2_Edu_21,Barratt_Barratt_P2_Occ_5,Barratt_Barratt_P2_Occ_10,Barratt_Barratt_P2_Occ_15,Barratt_Barratt_P2_Occ_20,Barratt_Barratt_P2_Occ_25,Barratt_Barratt_P2_Occ_30,Barratt_Barratt_P2_Occ_35,Barratt_Barratt_P2_Occ_40,Barratt_Barratt_P2_Occ_45
0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
cat_train_final = pd.concat([train_cat.drop(columns=columns_to_encode), train_encoded], axis=1)
cat_train_final.head()

,participant_id,Basic_Demos_Enroll_Year_2016,Basic_Demos_Enroll_Year_2017,Basic_Demos_Enroll_Year_2018,Basic_Demos_Enroll_Year_2019,Basic_Demos_Enroll_Year_2020,Basic_Demos_Study_Site_2,Basic_Demos_Study_Site_3,Basic_Demos_Study_Site_4,PreInt_Demos_Fam_Child_Ethnicity_1.0,...,Barratt_Barratt_P2_Edu_21,Barratt_Barratt_P2_Occ_5,Barratt_Barratt_P2_Occ_10,Barratt_Barratt_P2_Occ_15,Barratt_Barratt_P2_Occ_20,Barratt_Barratt_P2_Occ_25,Barratt_Barratt_P2_Occ_30,Barratt_Barratt_P2_Occ_35,Barratt_Barratt_P2_Occ_40,Barratt_Barratt_P2_Occ_45
0,UmrK0vMLopoR,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,CPaeQkhcjg7d,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Nb4EetVPm3gs,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,p4vPhVu91o4b,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M09PXs7arQ5E,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


## Function

In [ ]:
def impute_one_hot_categorical(cat, ethnicity_mode=None):
    """
    mode imputation for PreInt_Demos_Fam_Child_Ethnicity
    returns one-hot encoded df
    """
    if ethnicity_mode is None:
        ethnicity_mode = cat['PreInt_Demos_Fam_Child_Ethnicity'].mode()[0]
    cat.fillna({'PreInt_Demos_Fam_Child_Ethnicity':mode}, inplace=True)
    
    for col in cat.select_dtypes(include=['int', 'float']).columns:
        cat[col] = cat[col].astype('category')
    columns_to_encode = cat.columns[1:].tolist()
    cat_one_hot = pd.get_dummies(cat[columns_to_encode], drop_first=True)
    cat_one_hot = cat_one_hot.map(lambda x: 1 if x is True else (0 if x is False else x))
    cat = pd.concat([cat.drop(columns=columns_to_encode), cat_one_hot], axis=1)
    
    return cat, ethnicity_mode
    

In [ ]:
train_cat = pd.read_excel('../../Data/widsdatathon2025/TRAIN/TRAIN_CATEGORICAL_METADATA.xlsx')
train_cat.head()

,participant_id,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ,Barratt_Barratt_P2_Edu,Barratt_Barratt_P2_Occ
0,UmrK0vMLopoR,2016,1,0.0,0,1,21,45,21,45
1,CPaeQkhcjg7d,2019,3,1.0,2,3,15,15,0,0
2,Nb4EetVPm3gs,2016,1,1.0,8,1,18,40,0,0
3,p4vPhVu91o4b,2018,3,0.0,8,3,15,30,18,0
4,M09PXs7arQ5E,2019,3,0.0,1,3,15,20,0,0


In [ ]:
train_cat_prep, _ = impute_one_hot_categorical(train_cat)
train_cat_prep.head()

,participant_id,Basic_Demos_Enroll_Year_2016,Basic_Demos_Enroll_Year_2017,Basic_Demos_Enroll_Year_2018,Basic_Demos_Enroll_Year_2019,Basic_Demos_Enroll_Year_2020,Basic_Demos_Study_Site_2,Basic_Demos_Study_Site_3,Basic_Demos_Study_Site_4,PreInt_Demos_Fam_Child_Ethnicity_1.0,...,Barratt_Barratt_P2_Edu_21,Barratt_Barratt_P2_Occ_5,Barratt_Barratt_P2_Occ_10,Barratt_Barratt_P2_Occ_15,Barratt_Barratt_P2_Occ_20,Barratt_Barratt_P2_Occ_25,Barratt_Barratt_P2_Occ_30,Barratt_Barratt_P2_Occ_35,Barratt_Barratt_P2_Occ_40,Barratt_Barratt_P2_Occ_45
0,UmrK0vMLopoR,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,CPaeQkhcjg7d,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Nb4EetVPm3gs,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,p4vPhVu91o4b,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M09PXs7arQ5E,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


# 2. Quantitative

In [24]:
quantitative = pd.read_excel('../../Data/widsdatathon2025/TRAIN/TRAIN_QUANTITATIVE_METADATA.xlsx')
quantitative.head()

,participant_id,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
0,UmrK0vMLopoR,40.00,13,3,10,47,13,11,28,0,6,1,5,0,5,1,0,10,NaN
1,CPaeQkhcjg7d,-94.47,14,3,13,34,18,23,30,0,18,6,8,7,8,10,4,5,NaN
2,Nb4EetVPm3gs,-46.67,14,4,10,35,16,10,29,1,14,2,8,5,7,6,4,9,8.239904
3,p4vPhVu91o4b,-26.68,10,5,12,39,19,16,28,6,24,4,16,9,10,8,4,6,NaN
4,M09PXs7arQ5E,0.00,14,5,15,40,20,24,28,1,18,4,11,4,10,7,3,9,8.940679


## Imputation

In [25]:
print(quantitative.isna().any())

participant_id                False
EHQ_EHQ_Total                 False
ColorVision_CV_Score          False
APQ_P_APQ_P_CP                False
APQ_P_APQ_P_ID                False
APQ_P_APQ_P_INV               False
APQ_P_APQ_P_OPD               False
APQ_P_APQ_P_PM                False
APQ_P_APQ_P_PP                False
SDQ_SDQ_Conduct_Problems      False
SDQ_SDQ_Difficulties_Total    False
SDQ_SDQ_Emotional_Problems    False
SDQ_SDQ_Externalizing         False
SDQ_SDQ_Generating_Impact     False
SDQ_SDQ_Hyperactivity         False
SDQ_SDQ_Internalizing         False
SDQ_SDQ_Peer_Problems         False
SDQ_SDQ_Prosocial             False
MRI_Track_Age_at_Scan          True
dtype: bool


In [29]:
mean = quantitative['MRI_Track_Age_at_Scan'].mean()
print(mean)

11.245678424384526


In [30]:
quantitative.fillna({'MRI_Track_Age_at_Scan':mean}, inplace=True)

In [31]:
print(quantitative.isna().any())

participant_id                False
EHQ_EHQ_Total                 False
ColorVision_CV_Score          False
APQ_P_APQ_P_CP                False
APQ_P_APQ_P_ID                False
APQ_P_APQ_P_INV               False
APQ_P_APQ_P_OPD               False
APQ_P_APQ_P_PM                False
APQ_P_APQ_P_PP                False
SDQ_SDQ_Conduct_Problems      False
SDQ_SDQ_Difficulties_Total    False
SDQ_SDQ_Emotional_Problems    False
SDQ_SDQ_Externalizing         False
SDQ_SDQ_Generating_Impact     False
SDQ_SDQ_Hyperactivity         False
SDQ_SDQ_Internalizing         False
SDQ_SDQ_Peer_Problems         False
SDQ_SDQ_Prosocial             False
MRI_Track_Age_at_Scan         False
dtype: bool


## Function

In [32]:
def impute_quantitative(quant, mri_age_mean=None):
    """
    mean imputation for MRI_Track_Age_at_Scan
    """
    if mri_age_mean is None:
        mri_age_mean = quant['MRI_Track_Age_at_Scan'].mean()
    quant.fillna({'MRI_Track_Age_at_Scan':mri_age_mean}, inplace=True)
    
    return quant, mri_age_mean
    

In [34]:
quantitative = pd.read_excel('../../Data/widsdatathon2025/TRAIN/TRAIN_QUANTITATIVE_METADATA.xlsx')
print(quantitative.isna().any())

participant_id                False
EHQ_EHQ_Total                 False
ColorVision_CV_Score          False
APQ_P_APQ_P_CP                False
APQ_P_APQ_P_ID                False
APQ_P_APQ_P_INV               False
APQ_P_APQ_P_OPD               False
APQ_P_APQ_P_PM                False
APQ_P_APQ_P_PP                False
SDQ_SDQ_Conduct_Problems      False
SDQ_SDQ_Difficulties_Total    False
SDQ_SDQ_Emotional_Problems    False
SDQ_SDQ_Externalizing         False
SDQ_SDQ_Generating_Impact     False
SDQ_SDQ_Hyperactivity         False
SDQ_SDQ_Internalizing         False
SDQ_SDQ_Peer_Problems         False
SDQ_SDQ_Prosocial             False
MRI_Track_Age_at_Scan          True
dtype: bool


In [35]:
quantitative, _ = impute_quantitative(quantitative)
print(quantitative.isna().any())

participant_id                False
EHQ_EHQ_Total                 False
ColorVision_CV_Score          False
APQ_P_APQ_P_CP                False
APQ_P_APQ_P_ID                False
APQ_P_APQ_P_INV               False
APQ_P_APQ_P_OPD               False
APQ_P_APQ_P_PM                False
APQ_P_APQ_P_PP                False
SDQ_SDQ_Conduct_Problems      False
SDQ_SDQ_Difficulties_Total    False
SDQ_SDQ_Emotional_Problems    False
SDQ_SDQ_Externalizing         False
SDQ_SDQ_Generating_Impact     False
SDQ_SDQ_Hyperactivity         False
SDQ_SDQ_Internalizing         False
SDQ_SDQ_Peer_Problems         False
SDQ_SDQ_Prosocial             False
MRI_Track_Age_at_Scan         False
dtype: bool
